In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.backends.cudnn.benchmark = True
NB_CLASS=59

In [2]:
BATCH_SIZE=32
IMAGE_SIZE=224    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.densenet121_finetune(NB_CLASS)
    model.cuda()
    print('Done')
    return model

In [3]:
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet121/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.0
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet121/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
            print('loss has increased lr divide 10 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet121', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet121', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [4]:
train(60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/lujunfeng/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:02<00:00, 11128266.30it/s]


Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.109375,loss:3.912766
[epoch:0,batch:59]:acc: 0.228125,loss:3.528083
[epoch:0,batch:89]:acc: 0.303125,loss:3.221697
[epoch:0,batch:119]:acc: 0.357031,loss:2.988649
[epoch:0,batch:149]:acc: 0.396042,loss:2.811107
[epoch:0,batch:179]:acc: 0.433854,loss:2.642925
[epoch:0,batch:209]:acc: 0.465923,loss:2.496808
[epoch:0,batch:239]:acc: 0.488932,loss:2.383076
[epoch:0,batch:269]:acc: 0.506134,loss:2.284198
[epoch:0,batch:299]:acc: 0.519583,loss:2.201410
[epoch:0,batch:299]: val_loss:1.248647,val_acc:0.693985,val_total:4539
[epoch:0,batch:329]:acc: 0.532955,loss:2.122236
[epoch:0,batch:359]:acc: 0.544271,loss:2.054663
[epoch:0,batch:389]:acc: 0.554968,loss:1.991157
[epoch:0,batch:419]:acc: 0.564807,loss:1.935031
[epoch:0,batch:449]:acc: 0.574306,loss:1.883060
[epoch:0,batch:479]:acc: 0.580404,loss:1.839431
[epoch:0,batch:509]:acc: 0.585355,loss:1.797372
[epoch:0,batch:539]:acc: 0.592766,loss:1.758086
[epoch:0,batch:569]

[epoch:4,batch:119]:acc: 0.829427,loss:0.440178
[epoch:4,batch:149]:acc: 0.830625,loss:0.433789
[epoch:4,batch:179]:acc: 0.829861,loss:0.437938
[epoch:4,batch:209]:acc: 0.828423,loss:0.444028
[epoch:4,batch:239]:acc: 0.827214,loss:0.442134
[epoch:4,batch:269]:acc: 0.829514,loss:0.436706
[epoch:4,batch:299]:acc: 0.828125,loss:0.437437
[epoch:4,batch:299]: val_loss:0.447014,val_acc:0.827936,val_total:4539
[epoch:4,batch:329]:acc: 0.829451,loss:0.436036
[epoch:4,batch:359]:acc: 0.828125,loss:0.439168
[epoch:4,batch:389]:acc: 0.829327,loss:0.437786
[epoch:4,batch:419]:acc: 0.827902,loss:0.443789
[epoch:4,batch:449]:acc: 0.827361,loss:0.442603
[epoch:4,batch:479]:acc: 0.829232,loss:0.439954
[epoch:4,batch:509]:acc: 0.829596,loss:0.439197
[epoch:4,batch:539]:acc: 0.829340,loss:0.441492
[epoch:4,batch:569]:acc: 0.829550,loss:0.441020
[epoch:4,batch:599]:acc: 0.830313,loss:0.438130
[epoch:4,batch:599]: val_loss:0.450217,val_acc:0.825733,val_total:4539
[epoch:4,batch:629]:acc: 0.829812,loss:0.4

Process Process-159:
Traceback (most recent call last):
  File "/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 110, in _worker_loop
    data_queue.put((idx, samples))
  File "/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in put
    obj = _ForkingPickler.dumps(obj)
  File "/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
  File "/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 194, in reduce_storage
    df 

RuntimeError: DataLoader worker (pid 28821) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

In [4]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet121/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))   
    min_acc=max(modelParams['val_correct'],0.81)
    print('val_correct is %f'%(min_acc))
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet121/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
        optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
        print('lr now is %f'%(lr))
        print('now patience is %d '%(patience))
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))      
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet121', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet121', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
reuseTrain('../model/DesNet121/2018-11-04_acc_best.pth',60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :0.435168
val_correct is 0.845561
Epoch 7/59
----------
lr now is 0.000100
now patience is 0 
[epoch:7,batch:29]:acc: 0.842708,loss:0.358491
[epoch:7,batch:59]:acc: 0.851562,loss:0.333651
[epoch:7,batch:89]:acc: 0.861458,loss:0.315530
[epoch:7,batch:119]:acc: 0.861979,loss:0.314100
[epoch:7,batch:149]:acc: 0.866042,loss:0.306455
[epoch:7,batch:179]:acc: 0.867708,loss:0.306235
[epoch:7,batch:209]:acc: 0.868155,loss:0.306727
[epoch:7,batch:239]:acc: 0.865755,loss:0.307462
[epoch:7,batch:269]:acc: 0.866435,loss:0.304891
[epoch:7,batch:299]:acc: 0.865833,loss:0.305342
[epoch:7,batch:299]: val_loss:0.406287,val_acc:0.840934,val_total:4539
[epoch:7,batch:329]:acc: 0.865814,loss:0.305691
[epoch:7,batch:359]:acc: 0.865799,loss:0.306098
[epoch:7,batch:389]:acc: 0.864984,loss:0.307426
[epoch:7,batch:419]:acc: 0.864807,loss:0.308842
[epoch:7,batch:449]:acc: 0.865139,loss:0.308225
[epoch:7,batch:479]:acc: 0.864648,loss:0.308805
[epoch:7,batch:509]:acc: 0.865196,loss:0.308384
[epoc

[epoch:11,batch:29]:acc: 0.911458,loss:0.228214
[epoch:11,batch:59]:acc: 0.901563,loss:0.229808
[epoch:11,batch:89]:acc: 0.900694,loss:0.231198
[epoch:11,batch:119]:acc: 0.907292,loss:0.220020
[epoch:11,batch:149]:acc: 0.905833,loss:0.222408
[epoch:11,batch:179]:acc: 0.903472,loss:0.226043
[epoch:11,batch:209]:acc: 0.905208,loss:0.226821
[epoch:11,batch:239]:acc: 0.905859,loss:0.224206
[epoch:11,batch:269]:acc: 0.906713,loss:0.222694
[epoch:11,batch:299]:acc: 0.908750,loss:0.222007
[epoch:11,batch:299]: val_loss:0.352736,val_acc:0.862745,val_total:4539
[epoch:11,batch:329]:acc: 0.908807,loss:0.222475
[epoch:11,batch:359]:acc: 0.907813,loss:0.223383
[epoch:11,batch:389]:acc: 0.907452,loss:0.223969
[epoch:11,batch:419]:acc: 0.907515,loss:0.222870
[epoch:11,batch:449]:acc: 0.908264,loss:0.222162
[epoch:11,batch:479]:acc: 0.908268,loss:0.222868
[epoch:11,batch:509]:acc: 0.907843,loss:0.222624
[epoch:11,batch:539]:acc: 0.908218,loss:0.222448
[epoch:11,batch:569]:acc: 0.907730,loss:0.222596


lr now is 0.000001
now patience is 0 
[epoch:15,batch:29]:acc: 0.913542,loss:0.198884
[epoch:15,batch:59]:acc: 0.913542,loss:0.201259
[epoch:15,batch:89]:acc: 0.917014,loss:0.200795
[epoch:15,batch:119]:acc: 0.920312,loss:0.200175
[epoch:15,batch:149]:acc: 0.918958,loss:0.198547
[epoch:15,batch:179]:acc: 0.916667,loss:0.203799
[epoch:15,batch:209]:acc: 0.916220,loss:0.203786
[epoch:15,batch:239]:acc: 0.916016,loss:0.203271
[epoch:15,batch:269]:acc: 0.917130,loss:0.201940
[epoch:15,batch:299]:acc: 0.919792,loss:0.199075
[epoch:15,batch:299]: val_loss:0.351311,val_acc:0.862745,val_total:4539
[epoch:15,batch:329]:acc: 0.920360,loss:0.198618
[epoch:15,batch:359]:acc: 0.919965,loss:0.198359
[epoch:15,batch:389]:acc: 0.920112,loss:0.198221
[epoch:15,batch:419]:acc: 0.920238,loss:0.198653
[epoch:15,batch:449]:acc: 0.919514,loss:0.199560
[epoch:15,batch:479]:acc: 0.919466,loss:0.199674
[epoch:15,batch:509]:acc: 0.919975,loss:0.199075
[epoch:15,batch:539]:acc: 0.920775,loss:0.198056
[epoch:15,b

In [5]:
def TrainWithRawData(path,epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet121/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet121/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'DesNet121', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'DesNet121', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet121', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet121', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/DesNet121/2018-11-04_acc_best.pth',60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :0.357364
val_correct is 0.864067
Epoch 15/59
----------
begin lr is  0.0001
[epoch:15,batch:29]:acc: 0.921875,loss:0.188780
[epoch:15,batch:59]:acc: 0.923958,loss:0.178278
[epoch:15,batch:89]:acc: 0.929861,loss:0.174021
[epoch:15,batch:119]:acc: 0.930208,loss:0.176278
[epoch:15,batch:149]:acc: 0.929167,loss:0.178120
[epoch:15,batch:179]:acc: 0.928125,loss:0.179004
[epoch:15,batch:209]:acc: 0.926637,loss:0.179931
[epoch:15,batch:239]:acc: 0.928255,loss:0.176995
[epoch:15,batch:269]:acc: 0.929861,loss:0.174198
[epoch:15,batch:299]:acc: 0.930417,loss:0.172389
[epoch:15,batch:299]: val_loss:0.348654,val_acc:0.867151,val_total:4539
save new model loss,now loss is  0.3486535847187042
save new model acc,now acc is  tensor(0.8672, device='cuda:0')
[epoch:15,batch:329]:acc: 0.930303,loss:0.173446
[epoch:15,batch:359]:acc: 0.929427,loss:0.173616
[epoch:15,batch:389]:acc: 0.929808,loss:0.172369
[epoch:15,batch:419]:acc: 0.929241,loss:0.173667
[epoch:15,batch:449]:acc: 0.929722,l

[epoch:18,batch:929]:acc: 0.940020,loss:0.154580
[epoch:18,batch:959]:acc: 0.939779,loss:0.154669
[epoch:18,batch:989]:acc: 0.939615,loss:0.154977
[epoch:18] :acc: 0.939622,loss:0.155680,lr:0.000100,patience:2
[epoch:18]: val_loss:0.350762,val_acc:0.870015,
Epoch 19/59
----------
lr desencd
[epoch:19,batch:29]:acc: 0.937500,loss:0.151659
[epoch:19,batch:59]:acc: 0.938542,loss:0.160642
[epoch:19,batch:89]:acc: 0.939931,loss:0.156982
[epoch:19,batch:119]:acc: 0.938281,loss:0.158123
[epoch:19,batch:149]:acc: 0.937500,loss:0.158173
[epoch:19,batch:179]:acc: 0.938715,loss:0.155664
[epoch:19,batch:209]:acc: 0.939137,loss:0.155704
[epoch:19,batch:239]:acc: 0.941536,loss:0.153435
[epoch:19,batch:269]:acc: 0.941088,loss:0.154223
[epoch:19,batch:299]:acc: 0.940104,loss:0.155253
[epoch:19,batch:299]: val_loss:0.347381,val_acc:0.869354,val_total:4539
[epoch:19,batch:329]:acc: 0.939110,loss:0.155896
[epoch:19,batch:359]:acc: 0.938108,loss:0.157214
[epoch:19,batch:389]:acc: 0.938462,loss:0.155516
[e

[epoch:23,batch:59]:acc: 0.926562,loss:0.172204
[epoch:23,batch:89]:acc: 0.930556,loss:0.165305
[epoch:23,batch:119]:acc: 0.935937,loss:0.161015
[epoch:23,batch:149]:acc: 0.936458,loss:0.159638
[epoch:23,batch:179]:acc: 0.936285,loss:0.158988
[epoch:23,batch:209]:acc: 0.937202,loss:0.159062
[epoch:23,batch:239]:acc: 0.937760,loss:0.157852
[epoch:23,batch:269]:acc: 0.937269,loss:0.158313
[epoch:23,batch:299]:acc: 0.937187,loss:0.158488
[epoch:23,batch:299]: val_loss:0.345140,val_acc:0.870015,val_total:4539
[epoch:23,batch:329]:acc: 0.937784,loss:0.156945
[epoch:23,batch:359]:acc: 0.936979,loss:0.157560
[epoch:23,batch:389]:acc: 0.937019,loss:0.157505
[epoch:23,batch:419]:acc: 0.937351,loss:0.156847
[epoch:23,batch:449]:acc: 0.937778,loss:0.157508
[epoch:23,batch:479]:acc: 0.937370,loss:0.157793
[epoch:23,batch:509]:acc: 0.937561,loss:0.157687
[epoch:23,batch:539]:acc: 0.937442,loss:0.158394
[epoch:23,batch:569]:acc: 0.937390,loss:0.158650
[epoch:23,batch:599]:acc: 0.937865,loss:0.158186

[epoch:27,batch:269]:acc: 0.936921,loss:0.159929
[epoch:27,batch:299]:acc: 0.936979,loss:0.159796
[epoch:27,batch:299]: val_loss:0.346204,val_acc:0.870015,val_total:4539
[epoch:27,batch:329]:acc: 0.936553,loss:0.161581
[epoch:27,batch:359]:acc: 0.936285,loss:0.160448
[epoch:27,batch:389]:acc: 0.935897,loss:0.160609
[epoch:27,batch:419]:acc: 0.936235,loss:0.160322
[epoch:27,batch:449]:acc: 0.936319,loss:0.160710
[epoch:27,batch:479]:acc: 0.936784,loss:0.159600
[epoch:27,batch:509]:acc: 0.936029,loss:0.161208
[epoch:27,batch:539]:acc: 0.936227,loss:0.160773
[epoch:27,batch:569]:acc: 0.937061,loss:0.160089
[epoch:27,batch:599]:acc: 0.936667,loss:0.160275
[epoch:27,batch:599]: val_loss:0.345521,val_acc:0.871558,val_total:4539
[epoch:27,batch:629]:acc: 0.937004,loss:0.159420
[epoch:27,batch:659]:acc: 0.937074,loss:0.158914
[epoch:27,batch:689]:acc: 0.936911,loss:0.158928
[epoch:27,batch:719]:acc: 0.936675,loss:0.159022
[epoch:27,batch:749]:acc: 0.936542,loss:0.158716
[epoch:27,batch:779]:ac

[epoch:31,batch:419]:acc: 0.936310,loss:0.159756
[epoch:31,batch:449]:acc: 0.936111,loss:0.159617
[epoch:31,batch:479]:acc: 0.936133,loss:0.159533
[epoch:31,batch:509]:acc: 0.936029,loss:0.159436
[epoch:31,batch:539]:acc: 0.936227,loss:0.159332
[epoch:31,batch:569]:acc: 0.936623,loss:0.158796
[epoch:31,batch:599]:acc: 0.936719,loss:0.158891
[epoch:31,batch:599]: val_loss:0.345388,val_acc:0.869795,val_total:4539
[epoch:31,batch:629]:acc: 0.937004,loss:0.158687
[epoch:31,batch:659]:acc: 0.937169,loss:0.158134
[epoch:31,batch:689]:acc: 0.937092,loss:0.157861
[epoch:31,batch:719]:acc: 0.937066,loss:0.158191
[epoch:31,batch:749]:acc: 0.936833,loss:0.159278
[epoch:31,batch:779]:acc: 0.936258,loss:0.159696
[epoch:31,batch:809]:acc: 0.936343,loss:0.159700
[epoch:31,batch:839]:acc: 0.936310,loss:0.159862
[epoch:31,batch:869]:acc: 0.936566,loss:0.159494
[epoch:31,batch:899]:acc: 0.936562,loss:0.159571
[epoch:31,batch:899]: val_loss:0.346752,val_acc:0.870456,val_total:4539
[epoch:31,batch:929]:ac

[epoch:35,batch:599]: val_loss:0.346072,val_acc:0.871117,val_total:4539
[epoch:35,batch:629]:acc: 0.938095,loss:0.155981
[epoch:35,batch:659]:acc: 0.938305,loss:0.155822
[epoch:35,batch:689]:acc: 0.938225,loss:0.156043
[epoch:35,batch:719]:acc: 0.937934,loss:0.156640
[epoch:35,batch:749]:acc: 0.937542,loss:0.157430
[epoch:35,batch:779]:acc: 0.937460,loss:0.157707
[epoch:35,batch:809]:acc: 0.937423,loss:0.157208
[epoch:35,batch:839]:acc: 0.937054,loss:0.157744
[epoch:35,batch:869]:acc: 0.937464,loss:0.157458
[epoch:35,batch:899]:acc: 0.937465,loss:0.157634
[epoch:35,batch:899]: val_loss:0.347609,val_acc:0.870897,val_total:4539
[epoch:35,batch:929]:acc: 0.936929,loss:0.158119
[epoch:35,batch:959]:acc: 0.936979,loss:0.158631
[epoch:35,batch:989]:acc: 0.937468,loss:0.157802
[epoch:35] :acc: 0.937478,loss:0.157896,lr:0.000000,patience:1
[epoch:35]: val_loss:0.348681,val_acc:0.870015,
Epoch 36/59
----------
[epoch:36,batch:29]:acc: 0.942708,loss:0.156154
[epoch:36,batch:59]:acc: 0.938021,los